In [1]:
import sqlite3
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import random
import numpy as np
from textblob import TextBlob
import csv

class AdvancedSupplyChainInventorySystem:
    def __init__(self, db_name='advanced_inventory.db', risk_config_path='/content/analysis_results.json'):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self.risk_config_path = risk_config_path
        self._create_tables()
        self._load_initial_inventory()
        self._load_risk_configuration()

    def _create_tables(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS inventory (
                product_id TEXT PRIMARY KEY,
                product_name TEXT,
                total_stock INTEGER,
                min_threshold INTEGER,
                max_capacity INTEGER,
                unit_price REAL,
                risk_factor REAL DEFAULT 0
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS damage_log (
                log_id INTEGER PRIMARY KEY,
                product_id TEXT,
                quantity_damaged INTEGER,
                damage_reason TEXT,
                timestamp DATETIME
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS transport_delays (
                delay_id INTEGER PRIMARY KEY,
                product_id TEXT,
                expected_delivery DATETIME,
                actual_delivery DATETIME,
                delay_reason TEXT
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS sales_log (
                sale_id INTEGER PRIMARY KEY,
                product_id TEXT,
                quantity_sold INTEGER,
                sale_timestamp DATETIME,
                sale_status TEXT
            )
        ''')

        self.conn.commit()

    def _load_initial_inventory(self):
        initial_inventory = [
            ('LIB001', 'Standard Lithium Battery', 5000, 1000, 10000, 50.0),
            ('LIB002', 'High-Capacity Battery', 3000, 500, 7000, 75.0),
            ('LIB003', 'EV Battery Module', 1500, 250, 4000, 200.0)
        ]

        for product in initial_inventory:
            try:
                self.cursor.execute('''
                    INSERT OR REPLACE INTO inventory
                    (product_id, product_name, total_stock, min_threshold, max_capacity, unit_price)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', product)
            except sqlite3.IntegrityError:
                print(f"Product {product[0]} already exists")

        self.conn.commit()

    def _load_risk_configuration(self):
        try:
            # Default risk weights
            default_weights = {
                'inventory_level': 0.3,
                'lead_time': 0.2,
                'news_sentiment': 0.3,
                'textual_risk': 0.2
            }

            # If using the JSON file with risk analysis articles
            with open(self.risk_config_path, 'r') as f:
                risk_data = json.load(f)

            # Extract sentiment scores and risk analysis for configuration
            sentiment_scores = [
                article['sentiment_analysis']['score']
                for article in risk_data
                if 'sentiment_analysis' in article
            ]

            # Calculate average sentiment as a risk factor
            avg_sentiment = sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0.5

            # Adjust weights based on sentiment
            self.risk_config = {
                'inventory_risk_weights': {
                    'inventory_level': 0.3,
                    'lead_time': 0.2,
                    'news_sentiment': avg_sentiment,
                    'textual_risk': 1 - avg_sentiment
                }
            }
        except (FileNotFoundError, json.JSONDecodeError):
            # Fallback to default configuration
            self.risk_config = {
                'inventory_risk_weights': default_weights
            }

    def predict_risk(self, inventory_level, lead_time, news_text, textual_risk):
        # Sentiment analysis for news
        sentiment_score = TextBlob(news_text).sentiment.polarity

        # Normalize input parameters
        norm_inventory = min(max(inventory_level / 5000, 0), 1)
        norm_lead_time = min(max(lead_time / 30, 0), 1)
        norm_sentiment = (sentiment_score + 1) / 2  # Map [-1, 1] to [0, 1]
        norm_textual_risk = min(max(textual_risk / 10, 0), 1)

        # Calculate weighted risk score
        weights = self.risk_config['inventory_risk_weights']
        risk_score = (
            (1 - norm_inventory) * weights['inventory_level'] +
            (1 - norm_lead_time) * weights['lead_time'] +
            (1 - norm_sentiment) * weights['news_sentiment'] +
            norm_textual_risk * weights['textual_risk']
        )

        # Assign risk label
        if risk_score > 0.7:
            return 'High', risk_score
        elif risk_score > 0.3:
            return 'Medium', risk_score
        else:
            return 'Low', risk_score

    def update_inventory(self, product_id, risk_label, risk_score):
        risk_multipliers = {'High': 0.5, 'Medium': 0.75, 'Low': 1.0}

        self.cursor.execute('''
            UPDATE inventory
            SET risk_factor = ?,
                total_stock = total_stock * ?
            WHERE product_id = ?
        ''', (risk_score, risk_multipliers[risk_label], product_id))

        self.conn.commit()

        return self.generate_inventory_alerts(product_id)

    def log_damage(self, product_id, quantity_damaged, damage_reason):
        timestamp = datetime.now()
        self.cursor.execute('''
            INSERT INTO damage_log
            (product_id, quantity_damaged, damage_reason, timestamp)
            VALUES (?, ?, ?, ?)
        ''', (product_id, quantity_damaged, damage_reason, timestamp))

        self.cursor.execute('''
            UPDATE inventory
            SET total_stock = total_stock - ?
            WHERE product_id = ?
        ''', (quantity_damaged, product_id))

        self.conn.commit()

    def log_transport_delay(self, product_id, expected_delivery, actual_delivery, delay_reason):
        self.cursor.execute('''
            INSERT INTO transport_delays
            (product_id, expected_delivery, actual_delivery, delay_reason)
            VALUES (?, ?, ?, ?)
        ''', (product_id, expected_delivery, actual_delivery, delay_reason))

        self.conn.commit()

    def log_sales(self, product_id, quantity_sold):
        timestamp = datetime.now()
        sale_status = 'Normal'

        # Check if sales are significantly lower than average
        self.cursor.execute('''
            SELECT AVG(quantity_sold) as avg_sales
            FROM sales_log
            WHERE product_id = ? AND sale_timestamp > ?
        ''', (product_id, timestamp - timedelta(days=30)))

        avg_sales = self.cursor.fetchone()[0]

        if avg_sales and quantity_sold < avg_sales * 0.5:
            sale_status = 'Reduced Sales'

        self.cursor.execute('''
            INSERT INTO sales_log
            (product_id, quantity_sold, sale_timestamp, sale_status)
            VALUES (?, ?, ?, ?)
        ''', (product_id, quantity_sold, timestamp, sale_status))

        self.conn.commit()
        return sale_status

    def generate_inventory_alerts(self, product_id=None):
        query = '''
            SELECT product_id, product_name, total_stock, min_threshold, max_capacity, risk_factor
            FROM inventory
        '''
        params = []

        if product_id:
            query += ' WHERE product_id = ?'
            params.append(product_id)

        self.cursor.execute(query, params)

        alerts = []
        for row in self.cursor.fetchall():
            product_id, name, stock, min_threshold, max_capacity, risk_factor = row

            if stock < min_threshold:
                alerts.append({
                    'type': 'LOW_STOCK',
                    'product_id': product_id,
                    'product_name': name,
                    'current_stock': stock,
                    'risk_factor': risk_factor
                })
            elif stock > max_capacity:
                alerts.append({
                    'type': 'OVERSTOCKED',
                    'product_id': product_id,
                    'product_name': name,
                    'current_stock': stock,
                    'risk_factor': risk_factor
                })

        return alerts

def export_tables_to_csv(conn):
    """Export all tables from the SQLite database to CSV files."""
    cursor = conn.cursor()

    # Get list of tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table_name in tables:
        table_name = table_name[0]

        # Fetch all data from the table
        cursor.execute(f"SELECT * FROM {table_name}")
        columns = [column[0] for column in cursor.description]

        # Write to CSV
        csv_filename = f"{table_name}_data.csv"
        with open(csv_filename, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)

            # Write headers
            csv_writer.writerow(columns)

            # Write data
            csv_writer.writerows(cursor.fetchall())

        print(f"Exported {table_name} to {csv_filename}")

def main():
    inventory_system = AdvancedSupplyChainInventorySystem()

    # Simulate comprehensive risk scenarios for each product
    scenarios = [
        {
            'product_id': 'LIB001',
            'inventory_level': 3500,
            'lead_time': 10,
            'news_text': "Market shows strong potential for standard lithium batteries with increasing electric vehicle adoption",
            'textual_risk': 3,
            'damage_scenario': {
                'quantity': 75,
                'reason': "Transportation packaging failure"
            },
            'transport_delay': {
                'expected_delivery': datetime.now(),
                'actual_delivery': datetime.now() + timedelta(days=3),
                'delay_reason': "Port congestion"
            },
            'sales_volume': 350
        },
        {
            'product_id': 'LIB002',
            'inventory_level': 500,
            'lead_time': 25,
            'news_text': "High-capacity battery market facing supply chain disruptions",
            'textual_risk': 7,
            'damage_scenario': {
                'quantity': 50,
                'reason': "Warehouse climate control failure"
            },
            'transport_delay': {
                'expected_delivery': datetime.now(),
                'actual_delivery': datetime.now() + timedelta(days=5),
                'delay_reason': "Severe weather conditions"
            },
            'sales_volume': 100
        },
        {
            'product_id': 'LIB003',
            'inventory_level': 8000,
            'lead_time': 15,
            'news_text': "EV battery module market shows moderate stability",
            'textual_risk': 2,
            'damage_scenario': {
                'quantity': 25,
                'reason': "Minor handling damage"
            },
            'transport_delay': {
                'expected_delivery': datetime.now(),
                'actual_delivery': datetime.now() + timedelta(days=1),
                'delay_reason': "Minor routing adjustment"
            },
            'sales_volume': 250
        }
    ]

    # Comprehensive simulation of risk management scenarios
    for scenario in scenarios:
        print(f"\n--- Scenario for {scenario['product_id']} ---")

        # Risk Prediction
        risk_label, risk_score = inventory_system.predict_risk(
            inventory_level=scenario['inventory_level'],
            lead_time=scenario['lead_time'],
            news_text=scenario['news_text'],
            textual_risk=scenario['textual_risk']
        )
        print(f"Risk Assessment: {risk_label} (Score: {risk_score:.2f})")

        # Inventory Update
        alerts = inventory_system.update_inventory(scenario['product_id'], risk_label, risk_score)
        print("Inventory Alerts:")
        for alert in alerts:
            print(f"- {alert['type']}: {alert['product_name']} (Stock: {alert['current_stock']}, Risk Factor: {alert['risk_factor']:.2f})")

        # Damage Logging
        inventory_system.log_damage(
            scenario['product_id'],
            scenario['damage_scenario']['quantity'],
            scenario['damage_scenario']['reason']
        )
        print(f"Logged damage: {scenario['damage_scenario']['quantity']} units due to {scenario['damage_scenario']['reason']}")

        # Transport Delay Logging
        inventory_system.log_transport_delay(
            scenario['product_id'],
            scenario['transport_delay']['expected_delivery'],
            scenario['transport_delay']['actual_delivery'],
            scenario['transport_delay']['delay_reason']
        )
        print(f"Transport Delay: {scenario['transport_delay']['delay_reason']} ({scenario['transport_delay']['expected_delivery']} → {scenario['transport_delay']['actual_delivery']})")

        # Sales Logging
        sales_status = inventory_system.log_sales(scenario['product_id'], scenario['sales_volume'])
        print(f"Sales Status: {sales_status} (Volume: {scenario['sales_volume']} units)")

    # Export tables to CSV
    export_tables_to_csv(inventory_system.conn)

    # Close the database connection
    inventory_system.conn.close()

if __name__ == "__main__":
    main()


--- Scenario for LIB001 ---
Risk Assessment: Medium (Score: 0.64)
Inventory Alerts:
Logged damage: 75 units due to Transportation packaging failure
Transport Delay: Port congestion (2025-01-24 09:20:03.490420 → 2025-01-27 09:20:03.490426)
Sales Status: Normal (Volume: 350 units)

--- Scenario for LIB002 ---
Risk Assessment: High (Score: 0.81)
Inventory Alerts:
Logged damage: 50 units due to Warehouse climate control failure
Transport Delay: Severe weather conditions (2025-01-24 09:20:03.490439 → 2025-01-29 09:20:03.490440)
Sales Status: Normal (Volume: 100 units)

--- Scenario for LIB003 ---
Risk Assessment: Medium (Score: 0.58)
Inventory Alerts:
Logged damage: 25 units due to Minor handling damage
Transport Delay: Minor routing adjustment (2025-01-24 09:20:03.490443 → 2025-01-25 09:20:03.490444)
Sales Status: Normal (Volume: 250 units)
Exported inventory to inventory_data.csv
Exported damage_log to damage_log_data.csv
Exported transport_delays to transport_delays_data.csv
Exported sa